##### Imports:

In [84]:
from utils import calculate_covariance_from_chroma, separate_for_training, calculate_mu_from_chroma, calculate_transition_probabilites, format_indiv_chroma, predict, get_unique_predicted, calculate_initial_probabilities, separate_last_chord, chord_distance_with_quality
import pickle
from chroma import get_chromagram, get_key
import pandas as pd
from tqdm import tqdm
import numpy as np
from hmmlearn import hmm
from sklearn.metrics import f1_score
import altair as alt

##### Steps:

1. Training / Testing Data Split
2. Create Chromagram from Training Data
3. Create HMM Initialization Components
    - Initial State Probabilities
    - Transition Probability Matrix
    - Mu Value
    - Emission Matrix
4. Create HMM Object
5. Fit / Train HMM

##### Training / Test Data Split:

In [85]:
# Load data and split into training and test
with open(r"dataset.pkl", 'rb') as data:
    midi_data:dict = pickle.load(data)

training_piece_names, test_piece_names = separate_for_training(midi_data, 0.8)
NOTES_NAMES =   ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
FULL_CHORD_LIST = [note + suffix for note in NOTES_NAMES for suffix in ['', 'm', 'dim']]

##### Create Chromagram from Training Data:

In [86]:
song_chromagrams = []
for song_name in tqdm(list(training_piece_names)):
    indiv_chroma = get_chromagram(song_name, midi_data)
    formatted = format_indiv_chroma(indiv_chroma)
    song_chromagrams.append(indiv_chroma)

chromagram = pd.concat(song_chromagrams)
chromagram.head(200)

  0%|          | 0/4609 [00:00<?, ?it/s]

100%|██████████| 4609/4609 [01:50<00:00, 41.75it/s] 


,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Chord Actual
0,0,135,0,0,57,0,0,0,55,0,0,59,Em
1,0,55,0,0,57,0,0,0,55,0,0,59,Em
2,0,55,0,72,57,0,0,0,55,0,0,59,Em
3,0,55,0,0,57,0,0,0,55,0,0,59,Em
4,0,0,0,0,0,0,80,0,0,0,0,59,Em
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,126,0,0,185,0,0,0,0,54,0,61,0,E
55,126,0,0,185,0,0,0,0,54,0,61,0,E
56,0,0,0,110,0,51,0,111,0,0,60,0,F#
57,0,0,0,110,0,51,0,111,0,0,60,0,F#


##### Create HMM Components:

###### Initial State Probabilities:

In [87]:
initial_state_probabilties = calculate_initial_probabilities(training_piece_names, midi_data)
initial_state_probabilties

C        0.000217
Cm       0.000000
Cdim     0.000000
C#       0.001085
C#m      0.058364
C#dim    0.000000
D        0.119983
Dm       0.000000
Ddim     0.000000
D#       0.000217
D#m      0.019527
D#dim    0.000000
E        0.119766
Em       0.060534
Edim     0.000000
F        0.000000
Fm       0.000000
Fdim     0.000000
F#       0.021046
F#m      0.021697
F#dim    0.000000
G        0.121067
Gm       0.000000
Gdim     0.000000
G#       0.004773
G#m      0.153396
G#dim    0.000000
A        0.020829
Am       0.000000
Adim     0.000000
A#       0.000434
A#m      0.000000
A#dim    0.000000
B        0.118030
Bm       0.159037
Bdim     0.000000
dtype: float64

###### Transition Matrix:

In [88]:
transition_prob_matrix = calculate_transition_probabilites(chromagram)
print(transition_prob_matrix)
assert np.allclose(transition_prob_matrix.sum(axis=1), 1), "Not all rows sum to 1"

              C        Cm      Cdim        C#       C#m     C#dim         D  \
C      0.860465  0.000000  0.000000  0.002114  0.000000  0.000000  0.002114   
Cm     0.027778  0.027778  0.027778  0.027778  0.027778  0.027778  0.027778   
Cdim   0.027778  0.027778  0.027778  0.027778  0.027778  0.027778  0.027778   
C#     0.000000  0.000000  0.000000  0.864640  0.002452  0.000000  0.007847   
C#m    0.000000  0.000000  0.000000  0.000000  0.858191  0.000000  0.001630   
C#dim  0.027778  0.027778  0.027778  0.027778  0.027778  0.027778  0.027778   
D      0.000178  0.000000  0.000000  0.000178  0.001956  0.000000  0.862097   
Dm     0.027778  0.027778  0.027778  0.027778  0.027778  0.027778  0.027778   
Ddim   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
D#     0.000000  0.000000  0.000000  0.002078  0.001385  0.000000  0.005540   
D#m    0.000000  0.000000  0.000000  0.009091  0.004091  0.000000  0.005000   
D#dim  0.027778  0.027778  0.027778  0.027778  0.027

###### Mu Value:

In [89]:
mu = calculate_mu_from_chroma(chromagram)
mu = mu.to_numpy()

###### Covariance Matrix:

In [90]:
# Example usage
covars = calculate_covariance_from_chroma(chromagram)
print("Covariances shape:", covars.shape)

Covariances shape: (36, 12, 12)


In [91]:
model = hmm.GaussianHMM(n_components=transition_prob_matrix.shape[0], covariance_type="diag")
model.startprob_ = initial_state_probabilties
model.transmat_ = transition_prob_matrix.values
model.means_ = mu.reshape(-1, 1)
model.covars_ = np.array([np.diag(cov_matrix) + 1e-6 for cov_matrix in covars]).reshape(-1, 12)
model.n_features = 36

In [92]:
custom_encoding = {}
for i, chord in enumerate(FULL_CHORD_LIST):
    custom_encoding[chord] = i

true_labels = []
predicted_labels = []
for song_name in tqdm(list(test_piece_names)):
    last_chord, chromagram_without_last_chord = separate_last_chord(get_chromagram(song_name, midi_data))
    if not chromagram_without_last_chord.empty:
        encoded_chromagram_without_last_chord = chromagram_without_last_chord['Chord Actual'].apply(lambda x: custom_encoding.get(x, -1)).values.reshape(-1, 1)
        preds = model.predict(encoded_chromagram_without_last_chord)
        prediction = preds[-1]
        predicted_labels.append(prediction)
        true_labels.append(custom_encoding.get(last_chord, -1))

f1 = f1_score(true_labels, predicted_labels, average='micro')
print(f"F1 Score: {f1}")

100%|██████████| 1153/1153 [00:34<00:00, 33.59it/s]

F1 Score: 0.1326973113616652


In [93]:
inverted_custom_encoding = {val:key for key, val in custom_encoding.items()}

true_chords = pd.Series(true_labels).apply(lambda x: inverted_custom_encoding[x])
predicted_chords = pd.Series(predicted_labels).apply(lambda x: inverted_custom_encoding[x])
song_names = pd.Series(test_piece_names)

frame_data = {
    'Song Name': song_names,
    'True Chord': true_chords,
    'Predicted Chord': predicted_chords
}
chord_distance_df = pd.DataFrame(frame_data)

chord_distance_df['Distance'] = pd.Series([chord_distance_with_quality(pred, true) for pred, true in zip(chord_distance_df['True Chord'], chord_distance_df['Predicted Chord'])])


chord_distance_df.head()


,Song Name,True Chord,Predicted Chord,Distance
0,Niko_Kotoulas__RhythmChordProg_1_Bm-A-D-G (vi-...,G,A,-2
1,Niko_Kotoulas_Sus4_G#m-B-F#-G#m (vi-I-V-vi).mid,G#m,A,0
2,Niko_Kotoulas_Melody_3_B-D#m-C#-E (I-iii-ii-IV...,E,A,-5
3,Niko_Kotoulas__RhythmChordProg_1_Bm-A-Bm-G (vi...,G,A,-2
4,Niko_Kotoulas_Melody_4_Bm-Em-F# (vi-ii-iii) - ...,F#,A,-3


In [94]:
num_test_songs = chord_distance_df['Song Name'].count()
print(chord_distance_df['Distance'].apply(np.abs).mean())

distance_bars = alt.Chart(chord_distance_df).mark_bar(
    binSpacing = 0.1,
    width=1
).encode(
    x=alt.X('Song Name:N', axis=alt.Axis(labels=False, ticks=True), sort='-y').title(f'Songs (n={num_test_songs})'),
    y=alt.Y('Distance:Q').title('Distance (Actual - Predicted)'),
    color=alt.Color('Distance:N').scale(scheme='bluepurple'),
    tooltip=['Song Name', 'True Chord', 'Predicted Chord', 'Distance'],
).properties(
    title='Distribution of the distance between Predicted and Actual chords for each Test Song'
)

distance_avg_line = alt.Chart(chord_distance_df).mark_rule(
    color='red',
).encode(
    y=alt.Y('mean(Distance):Q', title='')
)

combined_chart = alt.layer(
    distance_bars,
    distance_avg_line
).properties(
    width=1000
)
combined_chart

3.3278404163052904


alt.LayerChart(...)